# Amazon Aurora
- Amazon Aurora는 상용 데이터베이스 비용의 1/10으로 완전한 MySQL, PostgreSQL 호환성을 통해 전 세계적으로 탁월한 고성능 및 가용성을 제공합니다. Aurora는 MySQL의 5배, PostgreSQL의 3배 처리량을 제공합니다.
- MySQL 및 PostgreSQL과 호환되는 완전 관리형 관계형 데이터베이스 엔진
    - AWS에서 클라우드 환경에 최적화 된 엔진을 자체 개발
- 용량의 자동 증감 : 10GB부터 시작하여 10GB 단위로 용량 증가 (최대 128TB)
- 연산 능력 : 128vCPU와 메모리 1024Gib까지 증가 가능
- 데이터 분산 저장 : 각 AZ마다 2개의 데이터 복제본 저장, 최소 3개 이상의 AZ = 최소 6개의 복제본
    - 3개 이상을 잃어버리기 전엔 쓰기 능력 유지
    - 4개 이상을 잃어버리기 전에는 읽기 능력 유지
    - 손실된 복제본은 자가 치유 : 지속적으로 손실된 부분을 검사 후 복구
    - Quorum 모델 사용

## Amazon Aurora DB Clusters
- Aurora는 연산 담당 노드와(읽기, 쓰기) 스토리지 노드가 따로 존재
- 하나 이상의 DB 인스턴스와 데이터를 관리하는 Cluster 볼륨을 묶은 단위
    - Cluster 볼륨 : 데이터베이스의 저장 공간으로, 여러 가용영역에 걸쳐 데이터를 복제 분산 저장
- 구성
    - Primary(Write) DB Instance : Read/Write 모두 가능한 인스턴스
        - 클러스터당 하나
    - Aurora Replica(Reader DB Instance) : Cluster 볼륨에 접근 가능한 DB Instacne로, Readaks 지원
        - 클러스터당 최대 15개 보유 가능
- 인스턴스간에 Async 복제
- Writer가 죽을 경우 자동으로 Replica중 하나가 Writer로 Failover
    - 데이터 손실 없이 Failover 시 메인으로 승격 가능
    - 고가용성을 확보

## Aurora Global Database
- 전 세계의 모든 리전에서 1초 내의 지연시간으로 데이터 엑세스 가능
- 재해 복구 용도로 활용 가능
    - 유사 시 보조 리전 중 하나를 승격하여 메인으로 활용
    - 1초의 RPO(복구 목표 지점) : 얼만큼 데이터가 날라갔느냐
    - 1분 미만의 RTO(복구 목표 시간) : 복구에 얼마나 걸리냐

## 오로라의 백업
- 읽기 복제본(Read Replica) 지원, 오로라 레플리카랑 다른 개념임
    - MySQL DB의 Binary Log 복제(Binlog) - 읽기용 복제본 생성
    - 다른 리전에만 생성 가능
- RDS와 마찬가지로 자동/수동 백업 가능
    - 자동 백업의 경우 1~35일 동안 보관(S3에 보관)
    - 수동 백업(스냅샷) 가능
    - 백업 데이터를 복원할 경우, 다른 데이터베이스를 생성


## 오로라 데이터베이스 클로닝
- 기존의 데이터베이스에서 새로운 데이터베이스를 복제
    - 스냅샷을 통해 새로운 데이터베이스를 생성하는 것 보다 빠르고 저렴
- Copy-On-Write 프로토콜 사용
    - 기존 클러스터를 삭제해도 제대로 동작

## Backtrack
- 기존의 DB를 특정 시점으로 되돌리는 것 (새로운 DB가 아닌 기존 DB)
    - DB 관리의 실수를 쉽게 만회 가능
    - 새로운 DB를 생성하는 것 보다 훨씬 빠름
    - 앞 뒤로 시점을 이동할 수 있기 때문에 원하는 지점을 빠르게 찾을 수 있음
- Backtrack Window
    - Target Backtrack Window
        - 어느 시점 만큼 DB를 되돌리기 위한 데이터를 저장할 것인지 
        - 24시간으로 설정하면, 24시간 전으로 돌아갈 수 있는 로그들을 보관하게 됨
        - 지정한 시점 이전으로는 Backtrack 불가능
    - Actual Backtrack Window
        - 실제로 시간을 얼만큼 되돌릴지(Target Backtrack Window 범위 안에서 골라!)
        - Target Backtrack Window보다 작아야 함
- 활성화 시 시간당 DB의 변화를 저장
    - 저장 된 용량만큼 비용 지불
    - DB 변화가 많을수록 많은 로그 = 비용
    - DB로그가 너무 많아 Actural Backtrack Window가 Target Backtrack Window 보다 작을 경우 알람
        - 설정한 백트랙 범위를 온전히 충족하기에 로그가 너무 많아서 실제로는 그만큼 보관하지 못하므로, 알람을 발생시켜 운영자가 인지할 수 있도록 한다
- MySQL만 가능
    - 오로라 생성 시 Backtrack을 설정 한 DB만 Backtrack 가능
- 스냅샷을 복구하거나 클론을 통해 기능 활성화 가능
- 다운타임 존재 (서비스 중지 시간)

## 데모 - 백트랙
- 오로라를 프로비전하고 백트랙 확인
    - 1분 단위 레코드를 입력하고 원하는 시간으로 백트랙
- MySQL 워크벤치 사용

## 실습 순서
1. RDS 데이터베이스 생성
    - 오로라 선택
    - 개발/테스트 템플릿 선택
    - DB 클러스터 : my-aurora
    - 자격 증명 관리 : 자체 관리 // 마스터 암호 입력
    - 인스턴스 : 버스터블 클래스 //  t3.medium
    - 퍼블릭 엑세스 on
    - 추가 구성
        - 초기 데이터베이스 구성 : my_db
        - 역추적 활성화, 기간 2시간
2. Workbench 연결
    - Standard TCP/IP
    - Hostname : 라이터 엔드포인트 입력
3. my-db에 테이블 생성 : db_test
    - idx

